# スコアリングデータへのData Recipeの適用と、予測の実施
Driverless AI [Custom Recipe](https://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/custom-recipes.html)

In [1]:
import os
import shutil
import urllib
import datetime

import numpy as np
import pandas as pd

import driverlessai

In [78]:
driverlessai.__version__

'1.9.3'

In [7]:
# Driverless AIのuser nameとpasswordの読み込み
import json
with open('../idpass.json') as f:
    idpass = json.load(f)

In [8]:
# Driverless AIサーバーへの接続
dai = driverlessai.Client(address='http://3.91.96.189', username=idpass['id'], password=idpass['pass1930'])
dai

<class 'driverlessai._core.Client'> http://3.91.96.189

## Experimentの選択

In [9]:
# 接続先Driverless AIのExperiments
dai.experiments.list()

    | Type       | Key                                  | Name
----+------------+--------------------------------------+------------------
  0 | Experiment | eaae200e-0c7f-11ec-8e41-0242ac110002 | amazon_mulCate_1
  1 | Experiment | 2f48f7fc-ffb5-11eb-9071-0242ac110002 | uci_cc_optTest2
  2 | Experiment | 3abf4f32-ffb0-11eb-9071-0242ac110002 | uci_cc_optTest
  3 | Experiment | 35f57a7c-da11-11eb-9ecf-0242ac110002 | boston2
  4 | Experiment | e4e76344-d9f6-11eb-9ecf-0242ac110002 | credit1
  5 | Experiment | a5b5eab4-d3ad-11eb-a4fa-0242ac110002 | saimple1
  6 | Experiment | 8f6c7d7c-d3ad-11eb-a4fa-0242ac110002 | boston1

In [10]:
# 特定のExperimentの取得
experiment = dai.experiments.get(key='eaae200e-0c7f-11ec-8e41-0242ac110002')
type(experiment)

driverlessai._experiments.Experiment

In [11]:
# Experimentのデータ
experiment.datasets

{'train_dataset': <class 'Dataset'> efb0cd00-0c7e-11ec-8e41-0242ac110002 amazon_reviews_JP_MultiCategory_train_TOKEN,
 'validation_dataset': None,
 'test_dataset': <class 'Dataset'> ae85d7f2-0c7f-11ec-8e41-0242ac110002 amazon_reviews_JP_MultiCategory_test_TOKEN}

In [12]:
# ターゲット変数
experiment.settings['target_column']

'product_category'

In [13]:
# ターゲット変数のサマリ
print(experiment.datasets['train_dataset'].column_summaries()[experiment.settings['target_column']])

# 6水準のカテゴリカルデータ

--- product_category ---

  Toys|█████████████████
    PC|██████████████
 Other|████████████████████

Data Type: str
Logical Types: []
Datetime Format: 
Count: 15583
Missing: 0
Unique: 6
Freq: 5242


In [14]:
# Dropped Columns
experiment.settings['drop_columns']

['C1', 'review_id', 'product_id', 'product_title']

In [15]:
# Experiment結果のサマリ
experiment.summary()

Status: Complete
Experiment: amazon_mulCate_1 (eaae200e-0c7f-11ec-8e41-0242ac110002)
  Version: 1.9.3, 2021-09-03 06:29
  Settings: 2/2/7, seed=176118131, GPUs disabled
  Train data: amazon_reviews_JP_MultiCategory_train_TOKEN (15583, 5)
  Validation data: N/A
  Test data: [Test] (2751, 4)
  Target column: product_category (6-class)
System specs: Docker/Linux, 31 GB, 8 CPU cores, 0/0 GPU
  Max memory usage: 0.679 GB, 0 GB GPU
Recipe: AutoDL (7 iterations, 2 individuals)
  Validation scheme: stratified, 1 internal holdout
  Feature engineering: 134 features scored (58 selected)
Timing: MOJO latency: 0.57213 millis (672.4kB)
  Data preparation: 5.11 secs
  Shift/Leakage detection: 1.88 secs
  Model and feature tuning: 29.76 secs (6 models trained)
  Feature evolution: 9.17 secs (2 of 22 models trained)
  Final pipeline training: 44.67 secs (9 models trained)
  Python / MOJO scorer building: 42.35 secs / 13.66 secs
Validation score: LOGLOSS = 1.52847 (constant preds)
Validation score: LOG

## 予測の実施

In [16]:
# ローカルにあるスコアリングデータ
scoring_data = 'amazon_reviews_JP_MultiCategory_scoring.csv'
scoring_data_path = os.path.join('..', 'data', scoring_data)
scoring_data_path

'../data/amazon_reviews_JP_MultiCategory_scoring.csv'

In [17]:
# スコアリングデータのDriverless上で指定する名前（実行時点日時）
scoring_data_name = 'scoring_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
scoring_data_name

'scoring_20210906-073019'

In [18]:
# スコアリングデータをクライアント環境からDriveless AIへアップロード
data_to_predict = dai.datasets.create(data=scoring_data_path, 
                                                                    data_source='upload', 
                                                                    name=scoring_data_name, 
                                                                    force=True)

Complete 100.00% - [4/4] Computed stats for column review


In [19]:
data_to_predict

<class 'Dataset'> dba0e0ae-0e98-11ec-b41d-0242ac110002 scoring_20210906-073019

In [20]:
data_to_predict.name

'scoring_20210906-073019'

In [21]:
data_to_predict.head(3)

review_id,product_id,product_title,star_rating,helpful_votes,review_date,review
R14IFZ98UPM5YD,B009LGO88E,ニンテンドー3DS LL 専用拡張スライドパッド,1,11,2012-12-14,モンスターハンター3G用に購入<br />LL本体がそこそこ重量あるので、それ＋スライドパッドでかなり重くなる<br />普通に持っていても腕が疲れてくるので長時間のプレイには適さない<br />個人的にはスライドパッドがあるからといって操作が格段に良くなる訳でもなく、むしろ腕に余計な負担がかかってゲームに集中できずイライラするだけなので一日使っただけで箱に戻した。LL本体の重量が携帯機であるギリギリの重さなんだなと思った。
RRFAQCR7MBQSZ,B002KCNVMA,レゴ (LEGO) シティ 空港 3182,5,0,2012-08-18,6才の子供と一緒に作りました。<br />飛行機は思っていたよりも大きくて作って楽しめました。<br />ドアの部分が難しかったようですが、他の部分は子供がほとんど作ることができました。<br />空港も回転扉があったりチケットカウンターがあったりと結構リアルです。<br />作りごたえがあり、親子とも非常に満足しています。
R1NEOQWC8FGQC5,B00712Y28Q,figma METROID Other M サムス・アラン(ABS&PVC製塗装済み可動フィギュア),5,4,2012-07-08,まず、あの有名なサムス・アランがフィギュアとして実体化したことがとても嬉しく思います。<br />当時はあの屈強なパワードスーツに金髪美女がいるなんて思いませんでしたが、、、<br />このモデルはOther:Mのパワードスーツになりますね。<br /><br />まず塗装は全体的に非常に綺麗です。メタリック！<br />塗装のはみ出しなんかもありませんでした。<br />可動もかなり動くんではないでしょうか、常軌を逸したポーズ以外で作品中のポーズはだいたいこなせると思います。<br />あと関節部分もすごく自然な感じで目立たないのがGoodです！足先は曲がるので地面を蹴り走るサムスの躍動感が出て良いですし、<br />股の付け根は目立たないよう覆われています。<br /><br />しかし気になるのも股でしょうかね、股を開かせようとすると覆いの部分が干渉してすこし外れやすいことぐらいですかね。<br />股を開かせようとするポージングなんてあまりしませんが笑<br /><br />あとは銃口が未塗装なのが残念ですね、主要武器の肝心な部分が塗装されていないのは良くないですよ、<br />他がほぼ完璧なため目立ちます。<br />付属品も少なめですし銃口パーツを付けても良かったのでは？<br /><br />結論としては超カッコイイ！サムスファンならもちろん買うべきですよこれは！<br /><br />願わくばスーパーメトロイドのスーツも欲しいなぁ（フュージョンのスーツは･･･自分は好きですが･･･）<br /><br />あとはモーフボールをどう飾れば…笑


### Data Recipeの適用

#### Data Recipe（元データに加工を加えて、新しいデータを作成  ）
日本語テキストカラムをトークン化するデータレシピ: [tokenization_japanese.py](https://github.com/yukismd/Driverless_AI_Recipe/blob/main/Data_recipe/tokenization_japanese.py)
- `cols_to_tokenize`にカラム名をリストで指定

In [22]:
# Data Recipeのダウンロード
data_recipe_url = 'https://raw.githubusercontent.com/yukismd/Driverless_AI_Recipe/main/Data_recipe/tokenization_japanese.py'
data_recipe_path = os.path.join('..', 'tmp', 'tokenization_japanese.py')   # 保存先

urllib.request.urlretrieve(data_recipe_url, data_recipe_path)

('../tmp/tokenization_japanese.py',
 <http.client.HTTPMessage at 0x7f8f9057beb0>)

In [23]:
# tokenization_japanese.pyのコピーを作成（copied_data_recipe.py）
data_recipe_path_copy = os.path.join('..', 'tmp', 'copied_data_recipe.py')
print(data_recipe_path_copy)
shutil.copyfile(data_recipe_path, data_recipe_path_copy)

../tmp/copied_data_recipe.py


'../tmp/copied_data_recipe.py'

In [24]:
# トークン化を実施するカラム（学習データで用いたカラム名と一致している必要）
text_colms = ['review']

In [25]:
text_colms_to_replace = "cols_to_tokenize = " + str(text_colms)
text_colms_to_replace   # 置換後の文字列

"cols_to_tokenize = ['review']"

In [26]:
# copied_data_recipe.pyのcols_to_tokenizeの置き換え

with open(data_recipe_path_copy) as f:
    text = f.read()

text = text.replace("cols_to_tokenize = []", text_colms_to_replace)   #テキストの置換

with open(data_recipe_path_copy, mode="w") as f:
    f.write(text)

In [27]:
# 置換できているか確認
with open(data_recipe_path_copy, 'r') as f:
    print(f.read())

"""
Japanese text tokenization using janome package - https://mocobeta.github.io/janome/en/
"""
from typing import Union, List
from h2oaicore.data import CustomData
import datatable as dt
import numpy as np
import pandas as pd

cols_to_tokenize = ['review']  # Column name list to tokenize

_global_modules_needed_by_name = ["janome"]


class TokenizeJapanese(CustomData):

    @staticmethod
    def create_data(X: dt.Frame = None) -> Union[str, List[str],
                                                 dt.Frame, List[dt.Frame],
                                                 np.ndarray, List[np.ndarray],
                                                 pd.DataFrame, List[pd.DataFrame]]:
        # exit gracefully if method is called as a data upload rather than data modify
        if X is None:
            return []
        # Tokenize the chinese text
        from janome.tokenizer import Tokenizer
        t = Tokenizer(wakati=True)
        X = dt.Frame(X).to_pandas()
        # If no colu

In [28]:
# テキストカラムのトークン化後のスコアリングデータ名（Driverless上）
scoring_data_name_token = data_to_predict.name + '_token'
scoring_data_name_token

'scoring_20210906-073019_token'

[Dataset.modify_by_recipe](https://docs.h2o.ai/driverless-ai/pyclient/docs/html/objects.html#driverlessai._datasets.Dataset.modify_by_recipe)

In [29]:
# Data Recipeを適用
data_to_predict_token = data_to_predict.modify_by_recipe(
    recipe=data_recipe_path_copy,
    names=[scoring_data_name_token]
)

Complete                                                            


In [30]:
data_to_predict_token = data_to_predict_token[scoring_data_name_token]  # dictからDatasetオブジェクトを取り出す
data_to_predict_token

<class 'Dataset'> 32971090-0e99-11ec-b41d-0242ac110002 scoring_20210906-073019_token

In [31]:
data_to_predict_token.head(3)

# reviewにトークン化が実施されている

review_id,product_id,product_title,star_rating,helpful_votes,review_date,review
R14IFZ98UPM5YD,B009LGO88E,ニンテンドー3DS LL 専用拡張スライドパッド,1,11,2012-12-14,モンスター ハンター 3 G 用 に 購入 < br /> LL 本体 が そこそこ 重量 ある ので 、 それ ＋ スライド パッド で かなり 重く なる < br /> 普通 に 持っ て い て も 腕 が 疲れ て くる ので 長時間 の プレイ に は 適さ ない < br /> 個人 的 に は スライド パッド が ある から と いっ て 操作 が 格段 に 良く なる 訳 で も なく 、 むしろ 腕 に 余計 な 負担 が かかっ て ゲーム に 集中 でき ず イライラ する だけ な ので 一 日 使っ た だけ で 箱 に 戻し た 。 LL 本体 の 重量 が 携帯 機 で ある ギリギリ の 重 さ な ん だ な と 思っ た 。
RRFAQCR7MBQSZ,B002KCNVMA,レゴ (LEGO) シティ 空港 3182,5,0,2012-08-18,6 才 の 子供 と 一緒 に 作り まし た 。 < br /> 飛行機 は 思っ て い た より も 大きく て 作っ て 楽しめ まし た 。 < br /> ドア の 部分 が 難しかっ た よう です が 、 他 の 部分 は 子供 が ほとんど 作る こと が でき まし た 。 < br /> 空港 も 回転 扉 が あっ たり チケット カウンター が あっ たり と 結構 リアル です 。 < br /> 作り ごたえ が あり 、 親子 とも 非常 に 満足 し て い ます 。
R1NEOQWC8FGQC5,B00712Y28Q,figma METROID Other M サムス・アラン(ABS&PVC製塗装済み可動フィギュア),5,4,2012-07-08,まず 、 あの 有名 な サムス ・ アラン が フィギュア として 実体 化 し た こと が とても 嬉しく 思い ます 。 < br /> 当時 は あの 屈強 な パワードスーツ に 金髪 美女 が いる なんて 思い ませ ん でし た が 、 、 、 < br /> この モデル は Other : M の パワードスーツ に なり ます ね 。 < br />< br /> まず 塗装 は 全体 的 に 非常 に 綺麗 です 。 メタ リック ！ < br /> 塗装 の はみ出し なんか も あり ませ ん でし た 。 < br /> 可動 も かなり 動く ん で は ない でしょ う か 、 常軌 を 逸し た ポーズ 以外 で 作品 中 の ポーズ は だいたい こなせる と 思い ます 。 < br /> あと 関節 部分 も すごく 自然 な 感じ で 目立た ない の が Good です ！ 足 先 は 曲がる ので 地面 を 蹴り 走る サムス の 躍動 感 が 出 て 良い です し 、 < br /> 股 の 付け根 は 目立た ない よう 覆わ れ て い ます 。 < br />< br /> しかし 気 に なる の も 股 でしょ う か ね 、 股 を 開か せよ う と する と 覆い の 部分 が 干渉 し て すこし 外れ やすい こと ぐらい です か ね 。 < br /> 股 を 開か せよ う と する ポージング なんて あまり し ませ ん が 笑 < br />< br /> あと は 銃口 が 未 塗装 な の が 残念 です ね 、 主要 武器 の 肝心 な 部分 が 塗装 さ れ て い ない の は 良く ない です よ 、 < br /> 他 が ほぼ 完璧 な ため 目立ち ます 。 < br /> 付属 品 も 少なめ です し 銃口 パーツ を 付け て も 良かっ た の で は ？< br />< br /> 結論 として は 超 カッコイイ ！ サムス ファン なら もちろん 買う べき です よ これ は ！< br />< br /> 願 わく ば スーパーメトロイド の スーツ も 欲しい なぁ （ フュージョン の スーツ は ･･･ 自分 は 好き です が ･･･）< br />< br /> あと は モーフボール を どう 飾れ ば … 笑


### スコアリングの実施

In [32]:
# スコアリングの実施
dai_prediction = experiment.predict(dataset=data_to_predict_token)

Complete


In [33]:
dai_prediction

In [34]:
# クライアント上に結果をダウンロード
download_name = dai_prediction.download(dst_dir=os.path.join('..', 'tmp'), overwrite=True)   # DLとパス名の取得

Downloaded '../tmp/eaae200e-0c7f-11ec-8e41-0242ac110002_preds_f9661d86.csv'


In [35]:
# Driveless上のデータ（アップしたスコアリングデータ、トークン化したデータ）を削除
data_to_predict.delete()
data_to_predict_token.delete()

Driverless AI Server reported dataset dba0e0ae-0e98-11ec-b41d-0242ac110002 deleted.
Driverless AI Server reported dataset 32971090-0e99-11ec-b41d-0242ac110002 deleted.


## ローカルへの読み込み

In [36]:
# DLしたスコアリング結果
df_dai = pd.read_csv(download_name)
print(df_dai.shape)
df_dai.head()

(25, 6)


,product_category.Camera,product_category.PC,product_category.Shoes,product_category.Toys,product_category.Video Games,product_category.Watches
0,0.033033,0.657850,0.012718,0.034772,0.247328,0.014299
1,0.035455,0.080561,0.024357,0.843720,0.006359,0.009548
2,0.081445,0.087765,0.022668,0.308333,0.015241,0.484547
3,0.121096,0.251459,0.076027,0.500707,0.020909,0.029802
4,0.564591,0.281474,0.032015,0.087534,0.013452,0.020934


In [37]:
res_cols = df_dai.columns.to_list()
res_cols

['product_category.Camera',
 'product_category.PC',
 'product_category.Shoes',
 'product_category.Toys',
 'product_category.Video Games',
 'product_category.Watches']

In [38]:
prod_category = [i.replace('product_category.', '') for i in res_cols]
prod_category

['Camera', 'PC', 'Shoes', 'Toys', 'Video Games', 'Watches']

In [70]:
# 元データへの結果の結合
df_res = pd.read_csv(scoring_data_path)
print(df_res.shape)

df_res = pd.concat([df_res, df_dai], axis=1)
print(df_res.shape)

df_res.head(3)

(25, 7)
(25, 13)


,review_id,product_id,product_title,star_rating,helpful_votes,review_date,review,product_category.Camera,product_category.PC,product_category.Shoes,product_category.Toys,product_category.Video Games,product_category.Watches
0,R14IFZ98UPM5YD,B009LGO88E,ニンテンドー3DS LL 専用拡張スライドパッド,1,11,2012-12-14,モンスターハンター3G用に購入<br />LL本体がそこそこ重量あるので、それ＋スライドパッ...,0.033033,0.657850,0.012718,0.034772,0.247328,0.014299
1,RRFAQCR7MBQSZ,B002KCNVMA,レゴ (LEGO) シティ 空港 3182,5,0,2012-08-18,6才の子供と一緒に作りました。<br />飛行機は思っていたよりも大きくて作って楽しめました...,0.035455,0.080561,0.024357,0.843720,0.006359,0.009548
2,R1NEOQWC8FGQC5,B00712Y28Q,figma METROID Other M サムス・アラン(ABS&PVC製塗装済み可動フ...,5,4,2012-07-08,まず、あの有名なサムス・アランがフィギュアとして実体化したことがとても嬉しく思います。<br...,0.081445,0.087765,0.022668,0.308333,0.015241,0.484547


### 追加情報の付与

In [71]:
def get_ranking(series):
    ''' 「確率の高い順の商品カテゴリ」「確率を大きい順」のリストを返す
    required:  prod_category which is ['Camera', 'PC', 'Shoes', 'Toys', 'Video Games', 'Watches']
    '''
    series = series.reset_index(drop=True)
    # Ranked Product Category
    prod_ranking = [prod_category[i] for i in series.argsort()]
    prod_ranking.reverse()
    # Ranked Prediction Value
    prediction_ranking = [series[i] for i in series.argsort()]
    prediction_ranking.reverse()
    return (prod_ranking, prediction_ranking)

In [72]:
print( get_ranking(df_res.loc[0, res_cols]) )
print( get_ranking(df_res.loc[1, res_cols]) )
print( get_ranking(df_res.loc[2, res_cols]) )

(['PC', 'Video Games', 'Toys', 'Camera', 'Watches', 'Shoes'], [0.65785, 0.24732834, 0.03477205, 0.033033088, 0.014298992, 0.012717536])
(['Toys', 'PC', 'Camera', 'Shoes', 'Watches', 'Video Games'], [0.8437197, 0.08056103, 0.035455216, 0.024357103, 0.009547653, 0.006359275])
(['Watches', 'Toys', 'PC', 'Camera', 'Shoes', 'Video Games'], [0.4845471, 0.3083335, 0.08776479, 0.08144523, 0.022668106, 0.015241274])


In [73]:
# 結果データに対して適用
res_prod, res_prediction = [], []
for i in df_res[res_cols].apply(get_ranking, axis=1):
    res_prod.append(i[0])    # カテゴリーランキング
    res_prediction.append(i[1])  # 予測値ランキング

In [76]:
# 結果データに変数として付与
df_res['Rank1'] = np.array(res_prod)[:,0] 
df_res['Rank2'] = np.array(res_prod)[:,1]
df_res['Rank3'] = np.array(res_prod)[:,2]
df_res['Pred_Rank1'] = np.array(res_prediction)[:,0]
df_res['Pred_Rank2'] = np.array(res_prediction)[:,1]
df_res.shape

(25, 18)

In [77]:
df_res.head()

,review_id,product_id,product_title,star_rating,helpful_votes,review_date,review,product_category.Camera,product_category.PC,product_category.Shoes,product_category.Toys,product_category.Video Games,product_category.Watches,Rank1,Rank2,Rank3,Pred_Rank1,Pred_Rank2
0,R14IFZ98UPM5YD,B009LGO88E,ニンテンドー3DS LL 専用拡張スライドパッド,1,11,2012-12-14,モンスターハンター3G用に購入<br />LL本体がそこそこ重量あるので、それ＋スライドパッ...,0.033033,0.657850,0.012718,0.034772,0.247328,0.014299,PC,Video Games,Toys,0.657850,0.247328
1,RRFAQCR7MBQSZ,B002KCNVMA,レゴ (LEGO) シティ 空港 3182,5,0,2012-08-18,6才の子供と一緒に作りました。<br />飛行機は思っていたよりも大きくて作って楽しめました...,0.035455,0.080561,0.024357,0.843720,0.006359,0.009548,Toys,PC,Camera,0.843720,0.080561
2,R1NEOQWC8FGQC5,B00712Y28Q,figma METROID Other M サムス・アラン(ABS&PVC製塗装済み可動フ...,5,4,2012-07-08,まず、あの有名なサムス・アランがフィギュアとして実体化したことがとても嬉しく思います。<br...,0.081445,0.087765,0.022668,0.308333,0.015241,0.484547,Watches,Toys,PC,0.484547,0.308333
3,R1LAKF7QVAA7ZB,B005M2V2K8,Beanie Boos ビーニー ブーズ カシミア ネコ,4,0,2014-09-08,写真よりも全体的に顔のパーツが中央に寄った感じの物が届きました。<br />あまり可愛くない...,0.121096,0.251459,0.076027,0.500707,0.020909,0.029802,Toys,PC,Camera,0.500707,0.251459
4,R38WP9YSEVBCIG,B007G3T2XU,Canon GPSレシーバーGP-E2,5,29,2014-01-25,EOS KISS X6iにて使用してます。<br />オプションアイテムとしてどうなんだろう...,0.564591,0.281474,0.032015,0.087534,0.013452,0.020934,Camera,PC,Toys,0.564591,0.281474
